<a href="https://colab.research.google.com/github/hirajya/CCDEPLRL_EXERCISES_COM222/blob/main/Exercise8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 8

Instructions: Finetune a model to answer the questions below.

In [24]:
!pip install --upgrade transformers datasets evaluate

In [25]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

## A. Model finetuning

In [26]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = inputs.pop("overflow_to_sample_mapping")
    offset_mapping = inputs.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = inputs["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = inputs.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                start_positions.append(cls_index)
                end_positions.append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                start_positions.append(token_start_index - 1)

                while token_end_index >= 0 and offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                end_positions.append(token_end_index + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs



In [27]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import TrainingArguments


model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)


args = TrainingArguments(
    output_dir="qa-finetuned-model",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=500,
    report_to="none"
)
tokenized_squad = squad.map(
    preprocess_function,
    batched=True,
    remove_columns=squad.column_names
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_squad,
    tokenizer=tokenizer
)

trainer.train()


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-27-d520d4f0ccac>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,5.834200
20,5.534000
30,5.080900
40,4.648100
50,4.318100
60,4.302100
70,4.189100
80,3.873100
90,3.624100
100,3.575300


TrainOutput(global_step=2548, training_loss=1.4288060367575246, metrics={'train_runtime': 808.9926, 'train_samples_per_second': 25.187, 'train_steps_per_second': 3.15, 'total_flos': 1996640674689024.0, 'train_loss': 1.4288060367575246, 'epoch': 4.0})

## B. Question Set

In [35]:
model.save_pretrained("qa-finetuned-model")
tokenizer.save_pretrained("qa-finetuned-model")

('qa-finetuned-model/tokenizer_config.json',
 'qa-finetuned-model/special_tokens_map.json',
 'qa-finetuned-model/vocab.txt',
 'qa-finetuned-model/added_tokens.json',
 'qa-finetuned-model/tokenizer.json')

In [36]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="qa-finetuned-model", tokenizer="qa-finetuned-model")


Device set to use cuda:0


1. Question 1

In [37]:
result = qa_pipeline(
    question="What is the largest ocean on Earth?",
    context= "The Pacific Ocean is the largest and deepest of Earth's oceanic divisions. It extends from the Arctic Ocean in the north to the Southern Ocean in the south"
)
print(result["answer"])  # Expected: Pacific Ocean

Pacific Ocean


2. Question 2

In [38]:
result = qa_pipeline(
    question="What was Isaac Newton's nationality?",
    context= "Isaac Newton was an English mathematician, physicist, astronomer, and author who is widely recognised as one of the greatest mathematicians and physicists of all time"
)
print(result["answer"])  # Expected: English

mathematician, physicist, astronomer


3. Question 3

In [39]:
result = qa_pipeline(
    question="where is Mount Everest located?",
    context= "Mount Everest is Earth's highest mountain above sea level, located in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China."
)
print(result["answer"])  # Expected Answer: in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China

the Himalayas


4. Question 4

In [40]:
result = qa_pipeline(
    question="What do plants use to perform photosynthesis?",
    context= "Photosynthesis is the process by which green plants and some other organisms use sunlight to synthesize foods from carbon dioxide and water"
)
print(result["answer"])  # Expected Answer: Sunlight

sunlight


5. Question 5

In [41]:
result = qa_pipeline(
    question="When did Barack Obama serve as president?",
    context= "Barack Obama served as the 44th president of the United States from 2009 to 2017."
)
print(result["answer"]) # Expected Answer: from 2009 to 2017

2009 to 2017
